In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import sys

In [73]:
df = pd.read_csv("D:/big data/practice/sasol/Test.csv")

In [74]:
data_frame = df.copy()
data_frame.shape

(380127, 18)

In [75]:
data_frame.head()

,ID,region,tenure,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,zone1,zone2,mrg,regularity,top_pack,freq_top_pack
0,ID_LZ8K373W7EWV,Droidika,K > 24 month,2008.55,16.0,2010.26,670.09,17.0,10.0,247.0,17.0,70.0,NaN,NaN,NO,59,On net 200F=Unlimited _call24H,11.0
1,ID_04KNWHGV5783,Luminous,K > 24 month,2991.45,14.0,2991.45,997.01,17.0,NaN,99.0,100.0,84.0,NaN,NaN,NO,49,All-net 500F=2000F;5d,12.0
2,ID_CXS07G6VLB2U,Astral,K > 24 month,1581.20,8.0,1578.21,526.07,8.0,436.0,47.0,113.0,5.0,NaN,NaN,NO,13,Jokko_Daily,6.0
3,ID_N3NFJJVHS25K,Eclipsara,K > 24 month,2564.10,12.0,2777.78,926.07,13.0,0.0,135.0,181.0,NaN,NaN,NaN,NO,53,All-net 500F=2000F;5d,13.0
4,ID_R36J4ROT93E9,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,1,NaN,NaN


# EDA via Descriptive Statistics


In [76]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380127 entries, 0 to 380126
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                380127 non-null  object 
 1   region            230399 non-null  object 
 2   tenure            380127 non-null  object 
 3   Amount            247072 non-null  float64
 4   refill_frequency  247072 non-null  float64
 5   revenue           252754 non-null  float64
 6   arpu_segment      252754 non-null  float64
 7   frequency         252754 non-null  float64
 8   data_volume       193087 non-null  float64
 9   on_net            241613 non-null  float64
 10  Procuct_1         222897 non-null  float64
 11  Procuct_2         153126 non-null  float64
 12  zone1             29861 non-null   float64
 13  zone2             24076 non-null   float64
 14  mrg               380127 non-null  object 
 15  regularity        380127 non-null  int64  
 16  top_pack          22

In [77]:
data_frame.dtypes.sort_values(ascending=False)

ID                   object
region               object
tenure               object
top_pack             object
mrg                  object
zone2               float64
zone1               float64
Procuct_2           float64
Procuct_1           float64
on_net              float64
data_volume         float64
frequency           float64
arpu_segment        float64
revenue             float64
refill_frequency    float64
Amount              float64
freq_top_pack       float64
regularity            int64
dtype: object

In [78]:
# generate descriptive statistics for numerical variables
data_frame.describe()

# generate descriptive statistics for numerical and categorical variables
data_frame.describe(include = 'all')

,ID,region,tenure,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,zone1,zone2,mrg,regularity,top_pack,freq_top_pack
count,380127,230399,380127,247072.000000,247072.000000,252754.00000,252754.000000,252754.000000,193087.000000,241613.000000,222897.000000,153126.000000,29861.000000,24076.000000,380127,380127.000000,221348,221348.000000
unique,380127,14,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,102,NaN
top,ID_LZ8K373W7EWV,Mandalore,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,NaN,All-net 500F=2000F;5d,NaN
freq,1,90615,360350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,380127,NaN,56038,NaN
mean,NaN,NaN,NaN,2361.094384,11.519853,2350.29115,783.433060,13.957963,3346.095454,278.806310,95.193008,23.052088,8.164663,7.655300,NaN,28.081699,NaN,9.261584
std,NaN,NaN,NaN,3041.162190,13.242743,3074.82589,1024.940158,14.652660,12002.653047,874.253933,203.950776,65.042688,48.857605,33.021478,NaN,22.289094,NaN,12.268907
min,NaN,NaN,NaN,10.680000,1.000000,0.43000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,NaN,1.000000
25%,NaN,NaN,NaN,427.350000,2.000000,427.35000,142.310000,3.000000,0.000000,5.000000,7.000000,2.000000,0.000000,0.000000,NaN,6.000000,NaN,2.000000
50%,NaN,NaN,NaN,1282.050000,7.000000,1282.05000,427.350000,9.000000,259.000000,27.000000,29.000000,6.000000,1.000000,2.000000,NaN,24.000000,NaN,5.000000
75%,NaN,NaN,NaN,3141.030000,16.000000,3141.45000,1047.010000,19.000000,2884.000000,157.000000,99.000000,20.000000,3.000000,5.000000,NaN,51.000000,NaN,12.000000


# EDA & Data Preparation


In [79]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380127 entries, 0 to 380126
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                380127 non-null  object 
 1   region            230399 non-null  object 
 2   tenure            380127 non-null  object 
 3   Amount            247072 non-null  float64
 4   refill_frequency  247072 non-null  float64
 5   revenue           252754 non-null  float64
 6   arpu_segment      252754 non-null  float64
 7   frequency         252754 non-null  float64
 8   data_volume       193087 non-null  float64
 9   on_net            241613 non-null  float64
 10  Procuct_1         222897 non-null  float64
 11  Procuct_2         153126 non-null  float64
 12  zone1             29861 non-null   float64
 13  zone2             24076 non-null   float64
 14  mrg               380127 non-null  object 
 15  regularity        380127 non-null  int64  
 16  top_pack          22

In [80]:
data_frame['top_pack'].describe()

count                    221348
unique                      102
top       All-net 500F=2000F;5d
freq                      56038
Name: top_pack, dtype: object

In [81]:
data_frame.dtypes.sort_values(ascending=False)

ID                   object
region               object
tenure               object
top_pack             object
mrg                  object
zone2               float64
zone1               float64
Procuct_2           float64
Procuct_1           float64
on_net              float64
data_volume         float64
frequency           float64
arpu_segment        float64
revenue             float64
refill_frequency    float64
Amount              float64
freq_top_pack       float64
regularity            int64
dtype: object

Data Cleaning

In [82]:
data_frame = data_frame.drop(['ID', 'mrg'], axis=1)

In [83]:
# Assuming your DataFrame is named df
data_frame['tenure_letter'] = data_frame['tenure'].str.extract('([A-Z])')
data_frame['tenure_months'] = data_frame['tenure'].str.extract('(\d+)').astype(float)

# Drop the original 'tenure' column if needed
data_frame = data_frame.drop('tenure', axis=1)

In [84]:
data_frame.shape

(380127, 17)

In [85]:
mode_value = data_frame['region'].mode().iloc[0]
data_frame['region'].fillna(mode_value, inplace=True)
data_frame.shape

(380127, 17)

In [86]:
data_frame.head()

,region,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,zone1,zone2,regularity,top_pack,freq_top_pack,tenure_letter,tenure_months
0,Droidika,2008.55,16.0,2010.26,670.09,17.0,10.0,247.0,17.0,70.0,NaN,NaN,59,On net 200F=Unlimited _call24H,11.0,K,24.0
1,Luminous,2991.45,14.0,2991.45,997.01,17.0,NaN,99.0,100.0,84.0,NaN,NaN,49,All-net 500F=2000F;5d,12.0,K,24.0
2,Astral,1581.20,8.0,1578.21,526.07,8.0,436.0,47.0,113.0,5.0,NaN,NaN,13,Jokko_Daily,6.0,K,24.0
3,Eclipsara,2564.10,12.0,2777.78,926.07,13.0,0.0,135.0,181.0,NaN,NaN,NaN,53,All-net 500F=2000F;5d,13.0,K,24.0
4,Mandalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,K,24.0


# Data Preprocessing for model

In [87]:
cleaned_df = data_frame.copy()

In [88]:
cleaned_df.shape

(380127, 17)

In [89]:
cleaned_df.dtypes

region               object
Amount              float64
refill_frequency    float64
revenue             float64
arpu_segment        float64
frequency           float64
data_volume         float64
on_net              float64
Procuct_1           float64
Procuct_2           float64
zone1               float64
zone2               float64
regularity            int64
top_pack             object
freq_top_pack       float64
tenure_letter        object
tenure_months       float64
dtype: object

In [90]:
cleaned_df

,region,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,zone1,zone2,regularity,top_pack,freq_top_pack,tenure_letter,tenure_months
0,Droidika,2008.55,16.0,2010.26,670.09,17.0,10.0,247.0,17.0,70.0,NaN,NaN,59,On net 200F=Unlimited _call24H,11.0,K,24.0
1,Luminous,2991.45,14.0,2991.45,997.01,17.0,NaN,99.0,100.0,84.0,NaN,NaN,49,All-net 500F=2000F;5d,12.0,K,24.0
2,Astral,1581.20,8.0,1578.21,526.07,8.0,436.0,47.0,113.0,5.0,NaN,NaN,13,Jokko_Daily,6.0,K,24.0
3,Eclipsara,2564.10,12.0,2777.78,926.07,13.0,0.0,135.0,181.0,NaN,NaN,NaN,53,All-net 500F=2000F;5d,13.0,K,24.0
4,Mandalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,K,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380122,Stellaris,4273.50,19.0,4487.18,1495.73,20.0,NaN,60.0,208.0,80.0,NaN,NaN,54,All-net 500F=2000F;5d,20.0,K,24.0
380123,Mandalore,6901.71,41.0,7115.38,2371.79,41.0,7019.0,105.0,257.0,5.0,NaN,NaN,59,"Data: 200 F=100MB,24H",33.0,K,24.0
380124,Mandalore,NaN,NaN,42.31,14.10,1.0,0.0,1.0,1.0,NaN,NaN,NaN,43,NaN,NaN,K,24.0
380125,Luminous,2307.69,9.0,2264.53,754.70,13.0,7381.0,15.0,NaN,19.0,2.0,NaN,61,"Data:490F=1GB,7d",5.0,K,24.0


In [91]:

columns_to_convert = ['Amount', 'refill_frequency', 'revenue', 'arpu_segment', 'frequency',
                       'data_volume', 'on_net', 'Procuct_1', 'Procuct_2','zone1', 'zone2',
                       'freq_top_pack']

cleaned_df[columns_to_convert] = cleaned_df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

In [92]:
cleaned_df.dtypes

region               object
Amount              float64
refill_frequency    float64
revenue             float64
arpu_segment        float64
frequency           float64
data_volume         float64
on_net              float64
Procuct_1           float64
Procuct_2           float64
zone1               float64
zone2               float64
regularity            int64
top_pack             object
freq_top_pack       float64
tenure_letter        object
tenure_months       float64
dtype: object

In [93]:
cleaned_df.isnull().sum()

region                   0
Amount              133055
refill_frequency    133055
revenue             127373
arpu_segment        127373
frequency           127373
data_volume         187040
on_net              138514
Procuct_1           157230
Procuct_2           227001
zone1               350266
zone2               356051
regularity               0
top_pack            158779
freq_top_pack       158779
tenure_letter            0
tenure_months            0
dtype: int64

In [94]:
# Replace missing values with the mean of each column
cleaned_df['Amount'].fillna(cleaned_df['Amount'].mean(), inplace=True)
cleaned_df['refill_frequency'].fillna(cleaned_df['refill_frequency'].mean(), inplace=True)
cleaned_df['data_volume'].fillna(cleaned_df['data_volume'].mean(), inplace=True)
cleaned_df['on_net'].fillna(cleaned_df['on_net'].mean(), inplace=True)
cleaned_df['Procuct_1'].fillna(cleaned_df['Procuct_1'].mean(), inplace=True)
cleaned_df['Procuct_2'].fillna(cleaned_df['Procuct_2'].mean(), inplace=True)
cleaned_df['zone1'].fillna(cleaned_df['zone1'].mean(), inplace=True)
cleaned_df['zone2'].fillna(cleaned_df['zone2'].mean(), inplace=True)
cleaned_df['freq_top_pack'].fillna(cleaned_df['freq_top_pack'].mean(), inplace=True)

cleaned_df['revenue'].fillna(cleaned_df['revenue'].mean(), inplace=True)
cleaned_df['arpu_segment'].fillna(cleaned_df['arpu_segment'].mean(), inplace=True)
cleaned_df['frequency'].fillna(cleaned_df['frequency'].mean(), inplace=True)

In [95]:

# Calculate the mode excluding NaN values
mode_value = cleaned_df['top_pack'].mode().iloc[0]

# Replace NaN values with mode_value
cleaned_df['top_pack'].fillna(mode_value, inplace=True)

# Replace empty strings with mode_value
cleaned_df['top_pack'].replace('', mode_value, inplace=True)


In [96]:
mode_value = cleaned_df['region'].mode().iloc[0]
cleaned_df['region'].fillna(mode_value, inplace=True)

In [97]:
cleaned_df.isnull().sum()

region              0
Amount              0
refill_frequency    0
revenue             0
arpu_segment        0
frequency           0
data_volume         0
on_net              0
Procuct_1           0
Procuct_2           0
zone1               0
zone2               0
regularity          0
top_pack            0
freq_top_pack       0
tenure_letter       0
tenure_months       0
dtype: int64

In [98]:
cleaned_df.head(70)

,region,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,zone1,zone2,regularity,top_pack,freq_top_pack,tenure_letter,tenure_months
0,Droidika,2008.550000,16.000000,2010.26000,670.09000,17.000000,10.000000,247.00000,17.000000,70.000000,8.164663,7.6553,59,On net 200F=Unlimited _call24H,11.000000,K,24.0
1,Luminous,2991.450000,14.000000,2991.45000,997.01000,17.000000,3346.095454,99.00000,100.000000,84.000000,8.164663,7.6553,49,All-net 500F=2000F;5d,12.000000,K,24.0
2,Astral,1581.200000,8.000000,1578.21000,526.07000,8.000000,436.000000,47.00000,113.000000,5.000000,8.164663,7.6553,13,Jokko_Daily,6.000000,K,24.0
3,Eclipsara,2564.100000,12.000000,2777.78000,926.07000,13.000000,0.000000,135.00000,181.000000,23.052088,8.164663,7.6553,53,All-net 500F=2000F;5d,13.000000,K,24.0
4,Mandalore,2361.094384,11.519853,2350.29115,783.43306,13.957963,3346.095454,278.80631,95.193008,23.052088,8.164663,7.6553,1,All-net 500F=2000F;5d,9.261584,K,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Mandalore,2361.094384,11.519853,2350.29115,783.43306,13.957963,0.000000,1.00000,95.193008,23.052088,8.164663,7.6553,15,All-net 500F=2000F;5d,9.261584,H,15.0
66,Zodiacalia,1025.640000,6.000000,1025.64000,341.88000,6.000000,0.000000,151.00000,66.000000,0.000000,8.164663,7.6553,11,On net 200F=Unlimited _call24H,2.000000,K,24.0
67,Zodiacalia,3183.760000,18.000000,3137.61000,1045.73000,18.000000,3346.095454,49.00000,188.000000,12.000000,8.164663,7.6553,61,All-net 500F =2000F_AllNet_Unlimited,10.000000,K,24.0
68,Celestial,384.620000,3.000000,385.04000,128.21000,4.000000,3346.095454,91.00000,95.193008,2.000000,8.164663,7.6553,56,On net 200F=Unlimited _call24H,2.000000,K,24.0


Task 2: Perform feature engineering

In [99]:
final_df = cleaned_df.copy()

In [100]:
final_df.isnull().sum()

region              0
Amount              0
refill_frequency    0
revenue             0
arpu_segment        0
frequency           0
data_volume         0
on_net              0
Procuct_1           0
Procuct_2           0
zone1               0
zone2               0
regularity          0
top_pack            0
freq_top_pack       0
tenure_letter       0
tenure_months       0
dtype: int64

In [101]:
final_df

,region,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,zone1,zone2,regularity,top_pack,freq_top_pack,tenure_letter,tenure_months
0,Droidika,2008.550000,16.000000,2010.26000,670.09000,17.000000,10.000000,247.00000,17.000000,70.000000,8.164663,7.6553,59,On net 200F=Unlimited _call24H,11.000000,K,24.0
1,Luminous,2991.450000,14.000000,2991.45000,997.01000,17.000000,3346.095454,99.00000,100.000000,84.000000,8.164663,7.6553,49,All-net 500F=2000F;5d,12.000000,K,24.0
2,Astral,1581.200000,8.000000,1578.21000,526.07000,8.000000,436.000000,47.00000,113.000000,5.000000,8.164663,7.6553,13,Jokko_Daily,6.000000,K,24.0
3,Eclipsara,2564.100000,12.000000,2777.78000,926.07000,13.000000,0.000000,135.00000,181.000000,23.052088,8.164663,7.6553,53,All-net 500F=2000F;5d,13.000000,K,24.0
4,Mandalore,2361.094384,11.519853,2350.29115,783.43306,13.957963,3346.095454,278.80631,95.193008,23.052088,8.164663,7.6553,1,All-net 500F=2000F;5d,9.261584,K,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380122,Stellaris,4273.500000,19.000000,4487.18000,1495.73000,20.000000,3346.095454,60.00000,208.000000,80.000000,8.164663,7.6553,54,All-net 500F=2000F;5d,20.000000,K,24.0
380123,Mandalore,6901.710000,41.000000,7115.38000,2371.79000,41.000000,7019.000000,105.00000,257.000000,5.000000,8.164663,7.6553,59,"Data: 200 F=100MB,24H",33.000000,K,24.0
380124,Mandalore,2361.094384,11.519853,42.31000,14.10000,1.000000,0.000000,1.00000,1.000000,23.052088,8.164663,7.6553,43,All-net 500F=2000F;5d,9.261584,K,24.0
380125,Luminous,2307.690000,9.000000,2264.53000,754.70000,13.000000,7381.000000,15.00000,95.193008,19.000000,2.000000,7.6553,61,"Data:490F=1GB,7d",5.000000,K,24.0


In [102]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
# Assuming your DataFrame is named df
le = LabelEncoder()

# Enumerate 'tenure_letter' column
final_df['tenure_letter'] = le.fit_transform(final_df['tenure_letter'])

# Enumerate 'region' column
final_df['region'] = le.fit_transform(final_df['region'])

# Enumerate 'region' column
final_df['top_pack'] = le.fit_transform(final_df['top_pack'])


In [103]:

final_df['revenue_tenure_interaction'] = final_df['revenue'] * final_df['tenure_months']

final_df.loc[final_df['revenue_tenure_interaction']==np.inf, 'revenue_tenure_interaction'] = 0


In [104]:
final_df['refill_regular_interaction'] = final_df['refill_frequency'] * final_df['regularity']

final_df.loc[final_df['refill_regular_interaction']==np.inf, 'refill_regular_interaction'] = 0

In [105]:
# Assuming you have a DataFrame called 'df'
# Create a new feature indicating the ratio of on-net calls to total calls
final_df['on_net_ratio'] = final_df['on_net'] / (final_df['Procuct_1'] + final_df['Procuct_2'] + final_df['zone1'] + final_df['zone2'])

final_df.loc[final_df['on_net_ratio']==np.inf, 'on_net_ratio'] = 0

In [106]:
final_df['data_volume_ratio'] = final_df['data_volume'] / (final_df['Procuct_1'] + final_df['Procuct_2'] + final_df['zone1'] + final_df['zone2'])

final_df.loc[final_df['data_volume_ratio']==np.inf, 'data_volume_ratio'] = 0

In [107]:
# You can adjust the threshold based on your domain knowledge
threshold = 0.5

# Create binary indicators based on on_net_ratio and data_volume_ratio
final_df['high_ratio'] = (final_df['on_net_ratio'] > threshold) | (final_df['data_volume_ratio'] > threshold)


In [108]:
final_df = final_df.replace({True: 1, False: 0})

In [109]:
final_df

,region,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,...,regularity,top_pack,freq_top_pack,tenure_letter,tenure_months,revenue_tenure_interaction,refill_regular_interaction,on_net_ratio,data_volume_ratio,high_ratio
0,2,2008.550000,16.000000,2010.26000,670.09000,17.000000,10.000000,247.00000,17.000000,70.000000,...,59,75,11.000000,7,24.0,48246.240000,944.000000,2.402257,0.097257,1
1,6,2991.450000,14.000000,2991.45000,997.01000,17.000000,3346.095454,99.00000,100.000000,84.000000,...,49,12,12.000000,7,24.0,71794.800000,686.000000,0.495446,16.745551,1
2,0,1581.200000,8.000000,1578.21000,526.07000,8.000000,436.000000,47.00000,113.000000,5.000000,...,13,55,6.000000,7,24.0,37877.040000,104.000000,0.351218,3.258109,1
3,3,2564.100000,12.000000,2777.78000,926.07000,13.000000,0.000000,135.00000,181.000000,23.052088,...,53,12,13.000000,7,24.0,66666.720000,636.000000,0.613993,0.000000,1
4,7,2361.094384,11.519853,2350.29115,783.43306,13.957963,3346.095454,278.80631,95.193008,23.052088,...,1,12,9.261584,7,24.0,56406.987605,11.519853,2.079634,24.958744,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380122,12,4273.500000,19.000000,4487.18000,1495.73000,20.000000,3346.095454,60.00000,208.000000,80.000000,...,54,12,20.000000,7,24.0,107692.320000,1026.000000,0.197485,11.013415,1
380123,7,6901.710000,41.000000,7115.38000,2371.79000,41.000000,7019.000000,105.00000,257.000000,5.000000,...,59,18,33.000000,7,24.0,170769.120000,2419.000000,0.377943,25.264563,1
380124,7,2361.094384,11.519853,42.31000,14.10000,1.000000,0.000000,1.00000,1.000000,23.052088,...,43,12,9.261584,7,24.0,1015.440000,495.353658,0.025080,0.000000,0
380125,6,2307.690000,9.000000,2264.53000,754.70000,13.000000,7381.000000,15.00000,95.193008,19.000000,...,61,30,5.000000,7,24.0,54348.720000,549.000000,0.121116,59.597100,1


In [110]:
cols_to_scale = ['Amount', 'refill_frequency', 'revenue', 'arpu_segment', 'frequency',
                       'data_volume', 'on_net', 'Procuct_1', 'Procuct_2', 'zone1', 'zone2',
                       'freq_top_pack', 'revenue_tenure_interaction', 'refill_regular_interaction', 'on_net_ratio', 'data_volume_ratio' ]

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
final_df[cols_to_scale] = scaler.fit_transform(final_df[cols_to_scale])

In [111]:
final_df

,region,Amount,refill_frequency,revenue,arpu_segment,frequency,data_volume,on_net,Procuct_1,Procuct_2,...,regularity,top_pack,freq_top_pack,tenure_letter,tenure_months,revenue_tenure_interaction,refill_regular_interaction,on_net_ratio,data_volume_ratio,high_ratio
0,2,0.023204,0.103448,0.023402,0.023407,0.177778,0.000010,0.009221,0.002679,0.018878,...,59,75,0.025707,7,24.0,0.023405,0.104187,0.002901,0.000004,1
1,6,0.034620,0.089655,0.034827,0.034827,0.177778,0.003422,0.003696,0.015760,0.022654,...,49,12,0.028278,7,24.0,0.034829,0.075682,0.000598,0.000608,1
2,0,0.018241,0.048276,0.018371,0.018376,0.077778,0.000446,0.001755,0.017809,0.001348,...,13,55,0.012853,7,24.0,0.018374,0.011380,0.000424,0.000118,1
3,3,0.029656,0.075862,0.032339,0.032349,0.133333,0.000000,0.005040,0.028526,0.006217,...,53,12,0.030848,7,24.0,0.032342,0.070158,0.000741,0.000000,1
4,7,0.027299,0.072551,0.027361,0.027367,0.143977,0.003422,0.010409,0.015003,0.006217,...,1,12,0.021238,7,24.0,0.027364,0.001162,0.002511,0.000907,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380122,12,0.049510,0.124138,0.052243,0.052248,0.211111,0.003422,0.002240,0.032782,0.021575,...,54,12,0.048843,7,24.0,0.052246,0.113247,0.000238,0.000400,1
380123,7,0.080035,0.275862,0.082846,0.082850,0.444444,0.007178,0.003920,0.040504,0.001348,...,59,18,0.082262,7,24.0,0.082848,0.267153,0.000456,0.000918,1
380124,7,0.027299,0.072551,0.000488,0.000493,0.000000,0.000000,0.000037,0.000158,0.006217,...,43,12,0.021238,7,24.0,0.000490,0.054619,0.000030,0.000000,0
380125,6,0.026678,0.055172,0.026363,0.026363,0.133333,0.007549,0.000560,0.015003,0.005124,...,61,30,0.010283,7,24.0,0.026365,0.060546,0.000146,0.002165,1


In [112]:
final_df.isnull().sum()

region                        0
Amount                        0
refill_frequency              0
revenue                       0
arpu_segment                  0
frequency                     0
data_volume                   0
on_net                        0
Procuct_1                     0
Procuct_2                     0
zone1                         0
zone2                         0
regularity                    0
top_pack                      0
freq_top_pack                 0
tenure_letter                 0
tenure_months                 0
revenue_tenure_interaction    0
refill_regular_interaction    0
on_net_ratio                  0
data_volume_ratio             1
high_ratio                    0
dtype: int64

In [113]:
final_df['data_volume_ratio'].fillna(final_df['data_volume_ratio'].mean(), inplace=True)
final_df.isnull().sum()

region                        0
Amount                        0
refill_frequency              0
revenue                       0
arpu_segment                  0
frequency                     0
data_volume                   0
on_net                        0
Procuct_1                     0
Procuct_2                     0
zone1                         0
zone2                         0
regularity                    0
top_pack                      0
freq_top_pack                 0
tenure_letter                 0
tenure_months                 0
revenue_tenure_interaction    0
refill_regular_interaction    0
on_net_ratio                  0
data_volume_ratio             0
high_ratio                    0
dtype: int64

In [122]:
from sklearn import decomposition
pca = decomposition.PCA()
pca.n_components = 22
pca_data = pca.fit_transform(final_df)
pca_data.shape

(380127, 22)

# testing

In [64]:
import h5py
from tensorflow.keras.models import load_model

model_path = 'D:/big data/practice/sasol/model_20231126082800.h5'

# Load the model
loaded_model = load_model(model_path)

In [123]:
predictions =loaded_model.predict(pca_data)
predictions[:5]

11879/11879 [==============================] - 34s 3ms/step


array([[9.9569917e-01],
       [0.0000000e+00],
       [9.9740988e-01],
       [0.0000000e+00],
       [3.9957269e-26]], dtype=float32)

In [131]:
y_pred = []
for element in predictions:
    if element < 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [132]:
len(y_pred)

380127

In [133]:
df[['ID']].shape

(380127, 1)

In [134]:
submission = df[['ID']]
submission['Target'] = y_pred

C:\Users\ME\AppData\Local\Temp\ipykernel_3188\142095896.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [135]:
submission

,ID,Target
0,ID_LZ8K373W7EWV,0
1,ID_04KNWHGV5783,1
2,ID_CXS07G6VLB2U,0
3,ID_N3NFJJVHS25K,1
4,ID_R36J4ROT93E9,1
...,...,...
380122,ID_MGZSA4ICBD9O,1
380123,ID_XGDT520DFBLP,1
380124,ID_9RDB792PB5Q4,1
380125,ID_KDTJHH8L8Z13,1


In [138]:
submission.to_csv('D:/big data/practice/sasol/submission.csv', index=False)

In [139]:
# calculate the no. of churned vs. retained
print(submission['Target'].value_counts())

# calculate the proportion
print(submission['Target'].value_counts(normalize=True)*100)

Target
1    297436
0     82691
Name: count, dtype: int64
Target
1    78.246481
0    21.753519
Name: proportion, dtype: float64


In [114]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

In [115]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]